In [1]:
import os
import gc

import numpy as np
import pandas as pd

import anndata
import scanpy as sc

In [2]:
count_file = 'AIT21_multiome.h5ad'

input_path = '/home/xingjiepan/data/whole_brain/scRNA-seq/20230723/'
annotation_file = '/home/xingjiepan/data/whole_brain/scRNA-seq/20230723/AIT21_annotation.tsv'

annotation_df = pd.read_csv(annotation_file, sep='\t')
annotation_df['cl'] = annotation_df['cl'].astype(str)
subclass_map = {t[0] : t[1] for t in annotation_df[['cl', 'subclass_label']].itertuples(index=False)}

In [3]:
%%time
adata = sc.read_h5ad(os.path.join(input_path, count_file))
adata.obs['subclass_label'] = adata.obs['cl'].map(subclass_map)

# Filter out cells without subclass annotations or annotated as low quality
mask_not_na = ~adata.obs['subclass_label'].isna()
mask_not_lq = adata.obs['subclass_label'] != 'LQ'
adata = adata[mask_not_na & mask_not_lq].copy()
gc.collect()

CPU times: user 163 ms, sys: 131 ms, total: 294 ms
Wall time: 293 ms


351

In [4]:
%%time
# Generate an adata with raw count
adata_raw = anndata.AnnData(X=adata.layers['rawcount'], obs=adata.obs, var=adata.var)
adata_raw.write_h5ad(count_file, compression='gzip')
adata_raw

<timed exec>:2: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


CPU times: user 1.33 s, sys: 29.4 ms, total: 1.36 s
Wall time: 1.45 s


AnnData object with n_obs × n_vars = 1687 × 32285
    obs: 'library_prep', 'gene.counts.0', 'doublet_score', 'roi', 'umi.counts', 'method', 'sex', 'external_donor_name', 'age', 'medical_conditions', 'cl', 'subclass_label'